In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import json

/Applications/anaconda3/envs/py35/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
PARTN_NUM = 120 # number of participants
NWORDS = 40
df  = pd.read_csv('./output/Shipley_Trials.csv') # file from the real experiment of a real test-taker

In [3]:
df.sample(5)

,SubjectID,Application,LogTime,TaskName,SessionNumber,TaskVersion,TrialNumber,TrialPrompt,TrialAnswer,TrialAccuracy,ProcessTime(msec)
6,MT,BGCScience Ver2.6.6,24_06_27_16_17_30,Shipley,0,Shipley,7,PROBABLE,likely,1,3334.796
16,MT,BGCScience Ver2.6.6,24_06_27_16_17_30,Shipley,0,Shipley,17,SOLEMN,serious,1,14271.850
33,MT,BGCScience Ver2.6.6,24_06_27_16_17_30,Shipley,0,Shipley,34,DIVEST,pledge,0,3401.147
5,MT,BGCScience Ver2.6.6,24_06_27_16_17_30,Shipley,0,Shipley,6,MASSIVE,large,1,3534.330
8,MT,BGCScience Ver2.6.6,24_06_27_16_17_30,Shipley,0,Shipley,9,FASCINATE,enchant,1,8669.980


In [11]:
# Generate the random experimental data
# We are using The Good, The Bad and The Ugly to name the grpup of participants. 
# The Good: those performed above the average test-taker; 30 people 
# The Bad: those performed below the average test-taker; 30 people
# The Ugly: those performed as an average test-taker; 60 people

real_answers = df['TrialAccuracy'].tolist()  # answers from the real test-taker
bad, ugly, good = {}, {}, {}

id = 0

# Helper function to generate randomized process times
def generate_random_times(base_times, offset_func):
    return [np.random.normal(offset_func(time), 1000) for time in base_times]

# Helper function to generate randomized answers
def generate_random_answers(real_answers, condition_func):
    return [condition_func(k, real_answers[k]) for k in range(len(real_answers))]

# Precompute the 'ProcessTime(msec)' list
process_times = df['ProcessTime(msec)']

# For 'Good' test-takers
for _ in range(30):
    good[f'answ_{id}'] = generate_random_answers(real_answers, lambda k, val: val if real_answers[k] == 0 else 1)
    good[f'time_{id}'] = generate_random_times(process_times, lambda x: x - np.random.randint(0, 1000))
    id += 1

# For 'Average' test-takers
for _ in range(60):
    ugly[f'answ_{id}'] = [1 - k if k == 1 else real_answers[k] for k in np.random.randint(0, 2, size=NWORDS)]
    ugly[f'time_{id}'] = generate_random_times(process_times, lambda x: x)
    id += 1

# For 'Bad' test-takers
for _ in range(30):
    bad[f'answ_{id}'] = generate_random_answers(real_answers, lambda k, val: val if real_answers[k] == 1 else 0)
    bad[f'time_{id}'] = generate_random_times(process_times, lambda x: x + np.random.randint(0, 1000))
    id += 1

In [14]:
art_data = pd.DataFrame({'words': df['TrialPrompt']})
art_data = pd.concat([art_data, pd.DataFrame(good), pd.DataFrame(ugly), pd.DataFrame(bad)], axis=1)

In [16]:
art_data.sample(5)

,words,answ_0,time_0,answ_1,time_1,answ_2,time_2,answ_3,time_3,answ_4,...,answ_115,time_115,answ_116,time_116,answ_117,time_117,answ_118,time_118,answ_119,time_119
12,HILARITY,1,3949.874534,1,2955.908517,1,4077.724303,1,3634.599119,1,...,1,6622.128286,1,4295.253178,1,3829.683686,1,4670.686852,1,5536.997127
5,MASSIVE,1,3671.105089,1,3008.253464,1,3677.185729,1,6225.529932,1,...,1,4832.068680,1,3658.520635,1,3592.286989,1,2285.349983,1,3833.275709
28,PRISTINE,0,2725.233407,0,3194.620497,0,2778.173532,0,3417.284002,0,...,0,4521.078230,0,5199.734217,0,6506.016125,0,3456.329133,0,5247.808392
9,EVIDENT,1,8123.201077,1,5301.576862,1,8742.186866,1,6142.125899,1,...,1,7322.831189,1,9012.038754,1,5116.439146,1,5294.785664,1,8108.836501
15,INDICATE,0,3895.938708,0,3441.596152,0,3867.833995,0,5596.069026,0,...,0,6957.161757,0,5680.143815,0,6353.085669,0,5398.555965,0,7547.638676


# Calculate IRT data

In [ ]:
!pip install py-irt

In [ ]:
# Add IRT test data

answers = [f'answ_{i}' for i in range(120)]
answ_data = art_data[answers]

In [ ]:
# Create a .jsonlines file to run the IRT model

subjects = {f'id_{i}':list(art_data[f'answ_{i}']) for i in range(120)}
words = art_data['words']

df_subj = pd.DataFrame(subjects)

output_path = "./art_test.jsonlines"

with open(output_path, 'w') as file:

    for subject_id in df_subj.columns:
        # Create a dictionary for each subject
        responses = {words[i]: int(df_subj[subject_id].iloc[i]) for i in range(len(words))}
        subject_data = {
            'subject_id': subject_id,
            'responses': responses
        }
        # Write the dictionary as a JSON object to the file
        file.write(json.dumps(subject_data) + '\n')

In [ ]:
!py-irt train 4pl art_test.jsonlines test-4pl/ --lr 0.02 --epochs 1000

In [ ]:
# Define the path to the JSON file
json_file_path = './test-4pl/best_parameters.json'

# Read the JSON file into a dictionary
with open(json_file_path, 'r') as file:
    irt_param = json.load(file)

# Add IRT data to the experimental data

In [121]:
# Add IRT parameters to the data
art_data['difficulty'] = irt_param['diff']

In [ ]:
# Add the mean and standard deviation of the answers and time

answ_columns = [col for col in art_data.columns if col.startswith('answ_')]
art_data['answ_mean'] = art_data[answ_columns].mean(axis=1)
art_data['answ_sd'] = art_data[answ_columns].std(axis=1)

time_columns = [col for col in art_data.columns if col.startswith('temp_')]
art_data['temp_mean'] = art_data[time_columns].mean(axis=1)
art_data['temp_sd'] = art_data[time_columns].std(axis=1)

In [125]:
art_data.sample(10)

,words,answ_0,temp_0,answ_1,temp_1,answ_2,temp_2,answ_3,temp_3,answ_4,...,temp_117,answ_118,temp_118,answ_119,temp_119,difficulty,answ_mean,answ_sd,temp_mean,temp_sd
6,PROBABLE,1,3743.985120,1,2669.292283,1,3355.588489,1,2847.669334,1,...,2766.104565,1,4286.711332,1,2919.012568,-0.542214,0.616667,0.488237,3289.156353,1015.065401
24,SQUANDER,0,14061.242151,0,14523.530695,0,14047.189733,0,14387.547585,0,...,12597.594170,1,13618.903999,0,11780.579596,0.080087,0.316667,0.467127,13203.136044,1172.758417
32,INEXORABLE,1,14074.024598,1,12185.580468,0,13966.716540,0,13730.522875,1,...,13977.427918,1,12724.863647,1,12559.277352,-0.863469,0.666667,0.473381,13271.241671,1055.312346
18,MERIT,1,11092.399197,1,10364.845922,1,9347.410194,1,11784.177971,0,...,8992.573588,1,10322.316030,1,12366.838361,-0.651901,0.616667,0.488237,10458.156899,1037.304608
14,CAPTION,1,7044.935166,1,6547.813745,0,8155.621185,0,6206.697701,0,...,6825.126295,1,5780.036911,1,4308.542295,-0.380656,0.591667,0.493586,5993.667963,1001.781340
38,PEREGRINATE,0,12479.494700,0,11614.787428,0,9717.995443,0,11495.923553,0,...,9156.889065,1,8499.694636,0,9429.467221,0.658070,0.275000,0.448386,10640.416620,1104.637726
23,CORDIAL,0,7247.793893,1,9658.031571,1,6637.271073,0,7897.518088,1,...,6022.686815,1,6787.960973,1,6107.064479,-0.485330,0.616667,0.488237,7340.293861,1069.662962
8,FASCINATE,0,8968.534245,1,9412.391211,1,9410.439418,1,8212.168508,0,...,8563.121123,1,9185.916224,1,7545.172782,-0.282588,0.583333,0.495074,8770.552774,1144.825745
3,PARDON,0,5279.396894,0,4962.738216,1,4001.925247,1,5173.507860,1,...,3736.070406,1,5803.961696,1,4488.350684,-0.641950,0.625000,0.486153,4442.633764,1133.723160
33,DIVEST,0,3288.381980,0,4006.402983,0,4209.653087,0,3526.289267,0,...,4764.298172,1,3366.941243,1,3115.768299,0.125446,0.366667,0.483915,3414.850514,1043.103899


In [124]:
art_data.to_csv('./data/art_generated_data.csv', index=False)